In [22]:
import pandas as pd

In [23]:
reviews = pd.read_parquet('D:/escritorio/STEAM/parquet/reviews.parquet')
output = pd.read_parquet('D:/escritorio/STEAM/parquet/output_game.parquet')
items = pd.read_parquet('D:/escritorio/STEAM/parquet/item.parquet')

Preparamos los datos

In [24]:
output.rename(columns={'id': 'item_id'}, inplace=True)
output.head(2)

,genres,app_name,price,item_id,developer,release_year
88310,Action,Lost Summoner Kitty,4.99,761140,Kotoshiro,2018
88311,Free to Play,Ironbound,0.00,643980,Secret Level SRL,2018


Unimos los 3 dataframes

In [25]:
# Unir 'itemseda' y 'outputeda' usando 'item_id' como clave
merged_df = pd.merge(items, output, on='item_id', how='inner')

# Unir 'merged_df' y 'reviewseda' usando 'item_id' como clave
juntos = pd.merge(merged_df, reviews, on='item_id', how='inner')

# Mostrar el nuevo DataFrame
juntos.head(2)

,item_id,item_name,playtime_forever,items_count,user_id_x,genres,app_name,price,developer,release_year,posted_year,user_id_y,recommend,review,sentiment_analysis
0,70,Half-Life,0,109,Nikiad,Action,Half-Life,9.99,Valve,1998,2015,EizanAratoFujimaki,True,ADVERTENCIA:Este Análisis contiene posibles Sp...,2
1,70,Half-Life,0,109,Nikiad,Action,Half-Life,9.99,Valve,1998,2011,GamerFag,True,If you own any half life but not this one that...,0


Eliminamos datos de nuestra tabla para que se pueda deployar

In [26]:
# Determina el tamaño de la muestra deseada
tamaño_muestra = 500000

# Realiza el muestreo aleatorio
muestra = juntos.sample(n=tamaño_muestra, random_state=42)  # Puedes ajustar el valor de 'random_state' para obtener resultados reproducibles

Juntamos en una nueva columas las 2 columnas llamadas user_id_x y user_id_y. A una que se llame solamente user_id

In [27]:
muestra['user_id'] = muestra['user_id_x'].fillna(muestra['user_id_y'])

# Elimina las columnas user_id_x y user_id_y si es necesario
muestra.drop(columns=['user_id_x', 'user_id_y'], inplace=True)

muestra.head(2)

,item_id,item_name,playtime_forever,items_count,genres,app_name,price,developer,release_year,posted_year,recommend,review,sentiment_analysis,user_id
685189,4000,Garry's Mod,4974,20,Indie,Garry's Mod,9.99,Facepunch Studios,2006,2014,True,I recommend getting this game because it is su...,2,ignaciomena22
1067561,304930,Unturned,1822,15,Action,Unturned,0.00,Smartly Dressed Games,2017,2015,True,I've just started this game and already enjoyi...,2,76561198077820683


Consultamos si tenemos valores nullos o lugares vacios

In [28]:
print(muestra.isnull().sum())

item_id               0
item_name             0
playtime_forever      0
items_count           0
genres                0
app_name              0
price                 0
developer             0
release_year          0
posted_year           0
recommend             0
review                0
sentiment_analysis    0
user_id               0
dtype: int64


Eliminamos los valores duplicados

In [33]:
# Elimina las filas duplicadas del DataFrame
unidos = muestra.drop_duplicates()
# Mostrar el DataFrame sin valores duplicados
unidos.head(2)

,item_id,item_name,playtime_forever,items_count,genres,app_name,price,developer,release_year,posted_year,recommend,review,sentiment_analysis,user_id
685189,4000,Garry's Mod,4974,20,Indie,Garry's Mod,9.99,Facepunch Studios,2006,2014,True,I recommend getting this game because it is su...,2,ignaciomena22
1067561,304930,Unturned,1822,15,Action,Unturned,0.00,Smartly Dressed Games,2017,2015,True,I've just started this game and already enjoyi...,2,76561198077820683


In [30]:
print(unidos.isin(['', ' ', None]).any().any())

True


Procedemos a guardar en formato parquet

In [31]:
unidos.to_parquet('D:/escritorio/STEAM/parquet/engineer.parquet')